In [ ]:
# This notebook cleans up datasets that contain provincial infomation and integrate them with the Weibo dataset

In [ ]:
import pandas as pd
import numpy as np

# Weibo data

In [ ]:
weibo_df = pd.read_pickle('../all_tweets_cleaned_final.pkl')

In [ ]:
from datetime import datetime

weibo_df['created_at'] = pd.to_datetime(weibo_df['created_at'], format='mixed')

def get_year(date):
    year = date.year
    return year

weibo_df['year'] = weibo_df['created_at'].apply(get_year)
weibo_df['year'].value_counts()

In [ ]:
weibo_df = weibo_df[~weibo_df['province'].isna()]
weibo_df = weibo_df[['province', 'year']]

In [ ]:
prov_list = weibo_df['province'].unique()
rows = []

for province in prov_list:
    province_row = {'province': province}
    for year in range(2011, 2025):
        count = 0
        count += len(weibo_df[(weibo_df['province'] == province) & (weibo_df['year'] == year)])
        province_row[year] = count
    rows.append(province_row)

province_year_count = pd.DataFrame(rows)

In [ ]:
province_year_count.to_excel('../supplementary_data/weibo_province_year_count.xlsx')

# Combined_violence_data cleaning

In [ ]:
vio_df = pd.read_excel('../supplementary_data/combined_violence_data.xlsx')
vio_df.info()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print(vio_df['province'].value_counts().index)

In [ ]:
vio_df[vio_df['province'] == '-']['City']

In [ ]:
for i in range(len(vio_df['City'])):
    if (vio_df['province'].iloc[i] == '-') or (pd.isna(vio_df['province'].iloc[i])):
        vio_df['province'].iloc[i] = vio_df['City'].iloc[i]
        

In [ ]:
vio_df[vio_df['province'] == 'NaN']['City']

In [ ]:
# Create a mapping dictionary for standardization (generated by ChatGPT)
province_mapping = {
    '上海': '上海', '上海\t': '上海', '上海市': '上海',
    '北京': '北京', '北京市': '北京',
    '天津': '天津', '天津市': '天津',
    '重庆': '重庆', '重庆市': '重庆',
    '西藏自治区': '西藏', 
    '新疆维吾尔自治': '新疆',
    '内蒙古自治区': '内蒙古', '內蒙古自治区': '内蒙古', '内蒙古': '内蒙古', '内蒙古自治': '内蒙古',
    '黑龙江省': '黑龙江', '黑龙江': '黑龙江',
    '吉林省': '吉林', '吉林': '吉林', '长春': '吉林',
    '辽宁省': '辽宁', '辽宁': '辽宁',
    '河北省': '河北', '河北': '河北', '渭南': '陕西', '咸阳': '陕西',
    '山东省': '山东', '山东': '山东', '潍坊市': '山东',
    '河南省': '河南', '河南': '河南',
    '江苏省': '江苏', '江苏': '江苏', ' 江苏省': '江苏', '江苏省 无锡市': '江苏',
    '安徽省': '安徽', '安徽': '安徽', '合肥市': '安徽',
    '浙江省': '浙江', '浙江': '浙江',
    '福建省': '福建', '福建': '福建',
    '广东省': '广东', '广东': '广东',
    '广西壮族自治区': '广西', '广西': '广西',
    '湖南省': '湖南', '湖南': '湖南',
    '湖北省': '湖北', '湖北': '湖北',
    '江西省': '江西', '江西': '江西',
    '四川省': '四川', '四川': '四川', '成都市': '四川',
    '贵州省': '贵州', '贵州': '贵州',
    '云南省': '云南',
    '陕西省': '陕西', '陕西': '陕西', '陜西': '陕西', '\t陕西省': '陕西', '渭南': '陕西', '西安市': '陕西',
    '甘肃省': '甘肃', '甘肃': '甘肃',
    '山西省': '山西', '山西': '山西',
    '黑龙江、辽宁、陕西、河北、湖南、河南、四川、山西和江西等共2': 'unspecified',
    'many provinces but not specified': 'unspecified', 'many, not specified': 'unspecified',
    '-': 'unspecified', 'not specified + multiple': 'unspecified', 
    'Korea/Japan': 'foreign', 'Thailand': 'foreign',
    '哈尔滨': '黑龙江', '曹路镇': 'unspecified', '盟科右中旗': '内蒙古', '鄂尔多斯鄂前旗': '内蒙古',
    '呼和浩特': '内蒙古'
}
# Clean up the column by stripping spaces, tabs, newlines, and applying the mapping
vio_df['province_clean'] = vio_df['province'].str.strip().replace(province_mapping)


In [ ]:
vio_df[vio_df['province_clean'].isnull()][['province', 'City']]

In [ ]:
vio_df['province_clean'].value_counts()

In [ ]:
vio_df['hire'].value_counts()

In [ ]:
vio_df['hire'] = vio_df['hire'].replace('-', np.nan)
vio_df['hire'] = vio_df['hire'].replace('unspecified', np.nan)

In [ ]:
hired_violence = vio_df[vio_df['hire'] == 1]
hired_violence.info()

In [ ]:
# total violence cases by province
cases_by_province = vio_df['province_clean'].value_counts().reset_index()
cases_by_province.columns = ['province', 'total_violence_cases']
cases_by_province

In [ ]:
hired_cases_by_province = vio_df.groupby('province_clean')['hire'].sum().reset_index()
hired_cases_by_province.columns =['province', 'hired_cases']

In [ ]:
vio_df_by_province = pd.merge(cases_by_province, hired_cases_by_province, on = 'province')

In [ ]:
vio_df_by_province['hired_ratio'] = vio_df_by_province['hired_cases']/vio_df_by_province['total_violence_cases']

In [ ]:
vio_df_by_province.info()

In [ ]:
vio_df_by_province.to_pickle('../supplementary_data/combined_violence_by_province.pkl')

# Protest_data

In [ ]:
# step 1: concatenate all sheets (except for Covid data) into one dataframe
# Load the Excel file
file_path = "../supplementary_data/protest_data.xlsx"

# Read each sheet separately
sheet1 = pd.read_excel(file_path, sheet_name='CLB(Jan 2020-Jul 2024)')
sheet2 = pd.read_excel(file_path, sheet_name='FH (Dec 2022- Apr 2024) ')
sheet3 = pd.read_excel(file_path, sheet_name='RFA (Jan 2020-Jun 2024)')
sheet4 = pd.read_excel(file_path, sheet_name='WQW(Jan 2020-Jun 2024)')
sheet5 = pd.read_excel(file_path, sheet_name='Covid (June 2022-Jan 2023')

In [ ]:
cols_to_keep = ['ID', 'Date', 'Rural or Urban', 'Province', 'City', 'Primary Grievance', 'hire', 'Violence', 'Violent Actions']

In [ ]:
sheet1 = sheet1[cols_to_keep]
sheet1

In [ ]:
sheet2 = sheet2[cols_to_keep]
sheet2.info()

In [ ]:
sheet3 = sheet3[cols_to_keep]
sheet3.info()

In [ ]:
sheet4 = sheet4[cols_to_keep]
sheet4.info()

In [ ]:
cols_to_keep_s5 = ['Case ID', 'Date', 'Rural or Urban', 'Province', 'City', 'Primary Grievance', 'Violence', 'Violent Actions']
sheet5 = sheet5[cols_to_keep_s5]
sheet5.info()

In [ ]:
sheet5.rename(columns = {'Case ID': 'ID'}, inplace = True)
sheet5['hire'] = np.nan
sheet5.info()

In [ ]:
new_col_order = ['ID', 'Date', 'Rural or Urban', 'Province', 'City', 'Primary Grievance', 'hire', 'Violence', 'Violent Actions']
sheet5 = sheet5[new_col_order]
sheet5.info()

In [ ]:
rows_to_keep = sheet5['ID'].str.startswith('CP')
sheet5 = sheet5[rows_to_keep]

In [ ]:
# Concatenate all DataFrames
prot_df = pd.concat([sheet1, sheet2, sheet3, sheet4, sheet5], ignore_index=True)

In [ ]:
prot_df.info()

In [ ]:
prot_df['Province'].value_counts().index

In [ ]:

# Create a mapping dictionary for standardization
province_mapping = {
    '上海': '上海', '上海\t': '上海', '上海市': '上海',
    '北京': '北京', '北京市': '北京',
    '天津': '天津', '天津市': '天津',
    '重庆': '重庆', '重庆市': '重庆',
    '西藏自治区': '西藏', 
    '新疆维吾尔自治': '新疆',
    '内蒙古自治区': '内蒙古', '內蒙古自治区': '内蒙古', '内蒙古': '内蒙古', '内蒙古自治': '内蒙古',
    '黑龙江省': '黑龙江', '黑龙江': '黑龙江',
    '吉林省': '吉林', '吉林': '吉林', '长春': '吉林',
    '辽宁省': '辽宁', '辽宁': '辽宁',
    '河北省': '河北', '河北': '河北', '渭南': '陕西', '咸阳': '陕西',
    '山东省': '山东', '山东': '山东', '潍坊市': '山东',
    '河南省': '河南', '河南': '河南',
    '江苏省': '江苏', '江苏': '江苏', ' 江苏省': '江苏', '江苏省 无锡市': '江苏',
    '安徽省': '安徽', '安徽': '安徽', '合肥市': '安徽',
    '浙江省': '浙江', '浙江': '浙江',
    '福建省': '福建', '福建': '福建',
    '广东省': '广东', '广东': '广东',
    '广西壮族自治区': '广西', '广西': '广西',
    '湖南省': '湖南', '湖南': '湖南',
    '湖北省': '湖北', '湖北': '湖北',
    '江西省': '江西', '江西': '江西',
    '四川省': '四川', '四川': '四川', '成都市': '四川',
    '贵州省': '贵州', '贵州': '贵州',
    '云南省': '云南',
    '陕西省': '陕西', '陕西': '陕西', '陜西': '陕西', '\t陕西省': '陕西', '渭南': '陕西', '西安市': '陕西',
    '甘肃省': '甘肃', '甘肃': '甘肃',
    '山西省': '山西', '山西': '山西',
    '黑龙江、辽宁、陕西、河北、湖南、河南、四川、山西和江西等共2': 'unspecified',
    'many provinces but not specified': 'unspecified', 'many, not specified': 'unspecified',
    '-': 'unspecified', 'not specified + multiple': 'unspecified', 
    'Korea/Japan': 'foreign', 'Thailand': 'foreign',
    '哈尔滨': '黑龙江', '曹路镇': 'unspecified', '盟科右中旗': '内蒙古', '鄂尔多斯鄂前旗': '内蒙古',
    '呼和浩特': '内蒙古',
    "内蒙古自治区": "内蒙古",
    "广西壮族自治区": "广西",
    "新疆维吾尔自治区": "新疆",
    "西藏自治区": "西藏",
    "宁夏回族自治区": "宁夏",
    "香港特别行政区": "香港",
    "江蘇省": "江苏",  
    "重慶市": "重庆",  
    "哪门股": '内蒙古',
    '海南省': '海南',
    '青海省': '青海'}
# Clean up the column by stripping spaces, tabs, newlines, and applying the mapping
prot_df['Province'] = prot_df['Province'].replace(province_mapping)


In [ ]:
prot_df.to_pickle('../supplementary_data/protest_data_all.pkl')

In [ ]:
prot_df['Province'].value_counts()

In [ ]:
prot_df['Date']

In [ ]:
from datetime import datetime

prot_df['Date'] = pd.to_datetime(prot_df['Date'], format='mixed')

def get_year(date):
    year = date.year
    return year

prot_df['year'] = prot_df['Date'].apply(get_year)
prot_df['year'].value_counts()

In [ ]:
prov_list = prot_df['Province'].unique()
rows = []

for province in prov_list:
    province_row = {'province': province}
    for year in range(2020, 2025):
        count = 0
        count += len(prot_df[(prot_df['Province'] == province) & (prot_df['year'] == year)])
        province_row[year] = count
    rows.append(province_row)

province_year_count = pd.DataFrame(rows)

In [ ]:
province_year_count.to_excel('../supplementary_data/protest_by_province.xlsx')

In [ ]:
hired_violence = prot_df[(prot_df['hire'] == 1) & (prot_df['Violence'] == 1)]

In [ ]:
hired_vio_by_province = hired_violence.groupby('Province').count().reset_index()

In [ ]:
hired_vio_by_province = hired_vio_by_province[['Province', 'Violence']]
hired_vio_by_province.columns = ('province', 'num_of_hired_vio_in_protest')

In [ ]:
hired_vio_by_province.to_pickle('../supplementary_data/hired_violence_in_protest_data.pkl')

In [ ]:
hired_vio_by_province['num_of_hired_vio_in_protest'].sum()

# Provincial data cleaning

In [ ]:
re_df = pd.read_excel('../supplementary_data/real_estate_data.xlsx', skiprows = 1)

In [ ]:
re_df.info()

In [ ]:
re_df_19 = re_df[re_df['Year'] == 2019]

In [ ]:
re_df_19['Region'].values

In [ ]:
pinyin_to_chinese = {
    'Anhui': '安徽',
    'Beijing': '北京',
    'Chongqing': '重庆',
    'Fujian': '福建',
    'Gansu': '甘肃',
    'Guangdong': '广东',
    'Guangxi': '广西',
    'Guizhou': '贵州',
    'Hainan': '海南',
    'Hebei': '河北',
    'Heilongjiang': '黑龙江',
    'Henan': '河南',
    'Hubei': '湖北',
    'Hunan': '湖南',
    'Inner Mongolia': '内蒙古',
    'Jiangsu': '江苏',
    'Jiangxi': '江西',
    'Jilin': '吉林',
    'Liaoning': '辽宁',
    'Ningxia': '宁夏',
    'Qinghai': '青海',
    'Shaanxi': '陕西',
    'Shandong': '山东',
    'Shanghai': '上海',
    'Shanxi': '山西',
    'Sichuan': '四川',
    'Tianjin': '天津',
    'Tibet': '西藏',
    'Xinjiang': '新疆',
    'Yunnan': '云南',
    'Zhejiang': '浙江'
}
re_df_19['province'] = re_df_19['Region'].replace(pinyin_to_chinese)
re_df_19

In [ ]:
re_df_19.to_pickle('../supplementary_data/provincial_data_2019.pkl')

# Anti-corruption Cases(updated) Dataset

In [ ]:
crp_data = pd.read_excel('../supplementary_data/Anti-corruption Cases(updated).xlsx')

In [ ]:
crp_data.columns

In [ ]:
crp_2020 = crp_data[crp_data['Years '] == 2020]

In [ ]:
crp_2020.columns = ('province', 'year', 'number_of_corruption_cases')

In [ ]:
len(crp_2020['province'].value_counts())

In [ ]:
pinyin_to_chinese = {
    'Anhui': '安徽',
    'Beijing': '北京',
    'Chongqing': '重庆',
    'Fujian': '福建',
    'Gansu': '甘肃',
    'Guangdong': '广东',
    'Guangxi': '广西',
    'Guizhou': '贵州',
    'Hainan': '海南',
    'Hebei': '河北',
    'Heilongjiang': '黑龙江',
    'Henan': '河南',
    'Hubei': '湖北',
    'Hunan': '湖南',
    'Inner Mongolia': '内蒙古',
    'Jiangsu': '江苏',
    'Jiangxi': '江西',
    'Jilin': '吉林',
    'Liaoning': '辽宁',
    'Ningxia': '宁夏',
    'Qinghai': '青海',
    'Shaanxi': '陕西',
    'Shandong': '山东',
    'Shanghai': '上海',
    'Shanxi': '山西',
    'Sichuan': '四川',
    'Tianjin': '天津',
    'Tibet': '西藏',
    'Xinjiang': '新疆',
    'Yunnan': '云南',
    'Zhejiang': '浙江',
    'The Ningxia Hui Autonomous Region': '宁夏',
    'Tibet Autonomous Region': '西藏',
    'jiangxi': '江西',
    'the Nei Monggol Autonomous Region': '内蒙古'
    
}
crp_2020['province'] = crp_2020['province'].replace(pinyin_to_chinese)


In [ ]:
crp_2020[['province', 'number_of_corruption_cases']].to_pickle('../supplementary_data/corruption_data_2020.pkl')

# audit data_2019

In [ ]:
audit = pd.read_excel('../supplementary_data/audit_data.xlsx')

In [ ]:
audit.columns = ('province', 'number_of_audit_cases_2019')

In [ ]:
province_mapping = {
    '广东省': '广东', '河南省': '河南', '山东省': '山东', '陕西省': '陕西', 
    '江苏省': '江苏', '广东省': '广东', '河北省': '河北', '浙江省': '浙江', 
    '四川省': '四川', '河南': '河南', '山东': '山东', '福建省': '福建', 
    '河北': '河北', '湖南省': '湖南', '湖北省': '湖北', '辽宁省': '辽宁', 
    '北京市': '北京', '四川': '四川', '陕西': '陕西', '上海市': '上海', 
    '山西省': '山西', '吉林省': '吉林', '江苏': '江苏', '黑龙江省': '黑龙江', 
    '安徽省': '安徽', '内蒙古自治区': '内蒙古', '辽宁': '辽宁', '湖南': '湖南', 
    '广西壮族自治区': '广西', '湖北': '湖北', '重庆市': '重庆', '贵州省': '贵州', 
    '江西省': '江西', '甘肃省': '甘肃', '云南省': '云南', '浙江': '浙江', 
    '北京': '北京', '上海': '上海', '天津市': '天津', '广西': '广西', 
    '新疆维吾尔自治区': '新疆', '-': None, '山西': '山西', '江西': '江西', 
    '福建': '福建', '吉林': '吉林', '重庆': '重庆', '海南省': '海南', 
    '安徽': '安徽', '黑龙江': '黑龙江', '内蒙古': '内蒙古', '云南': '云南', 
    '宁夏回族自治区': '宁夏', '海南': '海南', '天津': '天津', '贵州': '贵州', 
    '甘肃': '甘肃', '青海省': '青海', '西藏自治区': '西藏', '新疆': '新疆', 
    '西藏': '西藏', '香港特别行政区': '香港', '宁夏': '宁夏', '青海': '青海', 
    '江蘇省': '江苏', '重慶市': '重庆', '哪门股': '内蒙古'
}
# Canonize the list
audit['province'] = audit['province'].replace(province_mapping)


In [ ]:
audit['province'].values

In [ ]:
audit.to_pickle('../supplementary_data/audit_data_2019.pkl')

# Governance data (2020-2023)

In [ ]:
gov_df = pd.read_excel('../supplementary_data/Governance Data (2020-23).xlsx', skiprows = 1)

In [ ]:
gov_df.columns

In [ ]:
gov_df_2020 = gov_df[gov_df['Year'] == 2020]

In [ ]:
pinyin_to_chinese = {
    'Anhui': '安徽',
    'Beijing': '北京',
    'Chongqing': '重庆',
    'Fujian': '福建',
    'Gansu': '甘肃',
    'Guangdong': '广东',
    'Guangxi': '广西',
    'Guizhou': '贵州',
    'Hainan': '海南',
    'Hebei': '河北',
    'Heilongjiang': '黑龙江',
    'Henan': '河南',
    'Hubei': '湖北',
    'Hunan': '湖南',
    'Inner Mongolia': '内蒙古',
    'Jiangsu': '江苏',
    'Jiangxi': '江西',
    'Jilin': '吉林',
    'Liaoning': '辽宁',
    'Ningxia': '宁夏',
    'Qinghai': '青海',
    'Shaanxi': '陕西',
    'Shandong': '山东',
    'Shanghai': '上海',
    'Shanxi': '山西',
    'Sichuan': '四川',
    'Tianjin': '天津',
    'Tibet': '西藏',
    'Xinjiang': '新疆',
    'Yunnan': '云南',
    'Zhejiang': '浙江',
    'The Ningxia Hui Autonomous Region': '宁夏',
    'Tibet Autonomous Region': '西藏',
    'jiangxi': '江西',
    'the Nei Monggol Autonomous Region': '内蒙古'
    
}
gov_df_2020['province'] = gov_df_2020['Region'].replace(pinyin_to_chinese)


In [ ]:
gov_df_2020.to_pickle('../supplementary_data/governance_data_2020.pkl')

# Land requisition data 2019

In [ ]:
land_df = pd.read_excel('../supplementary_data/land_requisition.xlsx')

In [ ]:
pinyin_to_chinese = {
    'Anhui': '安徽',
    'Beijing': '北京',
    'Chongqing': '重庆',
    'Fujian': '福建',
    'Gansu': '甘肃',
    'Guangdong': '广东',
    'Guangxi': '广西',
    'Guizhou': '贵州',
    'Hainan': '海南',
    'Hebei': '河北',
    'Heilongjiang': '黑龙江',
    'Henan': '河南',
    'Hubei': '湖北',
    'Hunan': '湖南',
    'Inner Mongolia': '内蒙古',
    'Jiangsu': '江苏',
    'Jiangxi': '江西',
    'Jilin': '吉林',
    'Liaoning': '辽宁',
    'Ningxia': '宁夏',
    'Qinghai': '青海',
    'Shaanxi': '陕西',
    'Shandong': '山东',
    'Shanghai': '上海',
    'Shanxi': '山西',
    'Sichuan': '四川',
    'Tianjin': '天津',
    'Tibet': '西藏',
    'Xinjiang': '新疆',
    'Yunnan': '云南',
    'Zhejiang': '浙江',
    'The Ningxia Hui Autonomous Region': '宁夏',
    'Tibet Autonomous Region': '西藏',
    'jiangxi': '江西',
    'the Nei Monggol Autonomous Region': '内蒙古'
    
}
land_df['province'] = land_df['province'].replace(pinyin_to_chinese)


In [ ]:
land_df.to_pickle('../supplementary_data/land_requisition_2019.pkl')